# expanding the SQuAD dataset

In [1]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser, pipeline
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

NUM_PREPROCESSING_WORKERS = 2

/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-de.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-es.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/stopwords-en.txt'>
  w = (w.strip() for w in w.read().split(","))
/home/sambeck/.pyenv/versions/3.6.13/lib/python3.6/site-packages/pattern/vector/__init__.py:284: Res

In [2]:
MODEL='google/electra-small-discriminator'
TASK='qa'
DATASET='squad'
MAX_LENGTH=128
TRAIN_MAX_SAMPLES=1000   # or None

### original dataset

In [3]:
SQUAD = datasets.load_dataset('squad')

Reusing dataset squad (/home/sambeck/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [4]:
SQUAD

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

### adapt checklist tooling to Huggingface SQuAD standard

In [ ]:
editor = Editor()

In [24]:
ret = editor.template({
    'question': 'Who is the most {adj}?',
    'context': '{male1} is {adj}, but {male2} is more {adj}.' 
    },
    labels={'text': ['{male2}']},
    adj=['great', 'awesome', 'excellent'],
    remove_duplicates=True)
print(ret.data[0])
print(ret.labels[0])
print()
print(ret.data[1])
print(ret.labels[1])
print()

ret.answers = ret.labels
del ret.labels
ret.question = [d['question'] for d in ret.data]
ret.context = [d['context'] for d in ret.data]
ret.id = []
ret.title = []

for i in range(len(ret.answers)):
    ret.answers[i]['answer_start'] = [ret.context[i].find(ret.answers[i]['text'][0])]
    ret.id.append(str(hex(hash(str(ret.data[i]))))[2:])
    ret.title.append('whoismore')
    
del ret.data
    

{'question': 'Who is the most great?', 'context': 'John is great, but William is more great.'}
{'text': ['William']}

{'question': 'Who is the most awesome?', 'context': 'John is awesome, but William is more awesome.'}
{'text': ['William']}



In [25]:
d = datasets.Dataset.from_dict(ret)

In [26]:
d.save_to_disk('./new_dataset')